# Initialization

* implement unifyGNN+Contrastive
* try different noisy and augmentation
* try against with/without unnoticeable 

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]: 

import imp
import time
import argparse
import numpy as np
import torch

from torch_geometric.datasets import Planetoid,Reddit2,Flickr,PPI


# from torch_geometric.loader import DataLoader
# from help_funcs import prune_unrelated_edge,prune_unrelated_edge_isolated
import scipy.sparse as sp

# Training settings
parser = argparse.ArgumentParser()
parser.add_argument('--debug', action='store_true',
        default=True, help='debug mode')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
parser.add_argument('--seed', type=int, default=10, help='Random seed.')
parser.add_argument('--model', type=str, default='GCN', help='model',
                    choices=['GCN','GAT','GraphSage','GIN'])
parser.add_argument('--dataset', type=str, default='Pubmed', 
                    help='Dataset',
                    choices=['Cora','Citeseer','Pubmed','PPI','Flickr','ogbn-arxiv','Reddit','Reddit2','Yelp'])
parser.add_argument('--train_lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=32,
                    help='Number of hidden units of backdoor model.')
parser.add_argument('--num_hidden', type=int, default=32,
                    help='Number of hidden units.')
parser.add_argument('--num_proj_hidden', type=int, default=32,
                    help='Number of hidden units in MLP.')
parser.add_argument('--thrd', type=float, default=0.5)
parser.add_argument('--target_class', type=int, default=0)
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')
parser.add_argument('--epochs', type=int,  default=200, help='Number of epochs to train benign and backdoor model.')
parser.add_argument('--trojan_epochs', type=int,  default=400, help='Number of epochs to train trigger generator.')
parser.add_argument('--inner', type=int,  default=1, help='Number of inner')
parser.add_argument('--temperature', type=float,  default=0.5, help='Temperature')
# backdoor setting
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')
parser.add_argument('--trigger_size', type=int, default=3,
                    help='tirgger_size')
parser.add_argument('--use_vs_number', action='store_true', default=True,
                    help="if use detailed number to decide Vs")
parser.add_argument('--vs_ratio', type=float, default=0,
                    help="ratio of poisoning nodes relative to the full graph")
parser.add_argument('--vs_number', type=int, default=160,
                    help="number of poisoning nodes relative to the full graph")
# defense setting
parser.add_argument('--defense_mode', type=str, default="none",
                    choices=['prune', 'isolate', 'none'],
                    help="Mode of defense")
parser.add_argument('--prune_thr', type=float, default=0.2,
                    help="Threshold of prunning edges")
parser.add_argument('--target_loss_weight', type=float, default=1,
                    help="Weight of optimize outter trigger generator")
parser.add_argument('--homo_loss_weight', type=float, default=0,
                    help="Weight of optimize similarity loss")
parser.add_argument('--homo_boost_thrd', type=float, default=0.5,
                    help="Threshold of increase similarity")
# attack setting
parser.add_argument('--dis_weight', type=float, default=1,
                    help="Weight of cluster distance")
parser.add_argument('--selection_method', type=str, default='cluster_degree',
                    choices=['loss','conf','cluster','none','cluster_degree'],
                    help='Method to select idx_attach for training trojan model (none means randomly select)')
parser.add_argument('--test_model', type=str, default='GCN',
                    choices=['GCN','GAT','GraphSage','GIN'],
                    help='Model used to attack')
parser.add_argument('--evaluate_mode', type=str, default='1by1',
                    choices=['overall','1by1'],
                    help='Model used to attack')
# GPU setting
parser.add_argument('--device_id', type=int, default=3,
                    help="Threshold of prunning edges")
# GRACE setting
parser.add_argument('--gpu_id', type=int, default=0)
parser.add_argument('--config', type=str, default="config.yaml")
## Contrasitve setting
parser.add_argument('--cl_lr', type=float, default=0.0002)
# parser.add_argument('--cl_num_hidden', type=int, default=128)
parser.add_argument('--cl_num_proj_hidden', type=int, default=32)
parser.add_argument('--cl_num_layers', type=int, default=2)
parser.add_argument('--cl_activation', type=str, default='relu')
parser.add_argument('--cl_base_model', type=str, default='GCNConv')
parser.add_argument('--cont_weight', type=float, default=100)
parser.add_argument('--add_edge_rate_1', type=float, default=0)
parser.add_argument('--add_edge_rate_2', type=float, default=0)
parser.add_argument('--drop_edge_rate_1', type=float, default=0)
parser.add_argument('--drop_edge_rate_2', type=float, default=0.5)
parser.add_argument('--drop_feat_rate_1', type=float, default=0)
parser.add_argument('--drop_feat_rate_2', type=float, default=0.5)
parser.add_argument('--tau', type=float, default=0.4)
parser.add_argument('--cl_num_epochs', type=int, default=0)
parser.add_argument('--cl_weight_decay', type=float, default=0.00001)
parser.add_argument('--cont_batch_size', type=int, default=0)
parser.add_argument('--noisy_level', type=float, default=0.25)

# args = parser.parse_args()
args = parser.parse_known_args()[0]
args.cuda =  not args.no_cuda and torch.cuda.is_available()
device = torch.device(('cuda:{}' if torch.cuda.is_available() else 'cpu').format(args.device_id))

np.random.seed(args.seed)
# torch.manual_seed(args.seed)
# torch.cuda.manual_seed(args.seed)
print(args)
#%%
from torch_geometric.utils import to_undirected
import torch_geometric.transforms as T
transform = T.Compose([T.NormalizeFeatures()])

if(args.dataset == 'Cora' or args.dataset == 'Citeseer' or args.dataset == 'Pubmed'):
    dataset = Planetoid(root='./data/', \
                        name=args.dataset,\
                        transform=transform)
elif(args.dataset == 'Flickr'):
    dataset = Flickr(root='./data/Flickr/', \
                    transform=transform)
elif(args.dataset == 'Reddit2'):
    dataset = Reddit2(root='./data/Reddit2/', \
                    transform=transform)
elif(args.dataset == 'ogbn-arxiv'):
    from ogb.nodeproppred import PygNodePropPredDataset
    # Download and process data at './dataset/ogbg_molhiv/'
    dataset = PygNodePropPredDataset(name = 'ogbn-arxiv', root='./data/')
    split_idx = dataset.get_idx_split() 

data = dataset[0].to(device)

if(args.dataset == 'ogbn-arxiv'):
    nNode = data.x.shape[0]
    setattr(data,'train_mask',torch.zeros(nNode, dtype=torch.bool).to(device))
    # dataset[0].train_mask = torch.zeros(nEdge, dtype=torch.bool).to(device)
    data.val_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.test_mask = torch.zeros(nNode, dtype=torch.bool).to(device)
    data.y = data.y.squeeze(1)
# we build our own train test split 
#%% 
from utils import get_split
data, idx_train, idx_val, idx_clean_test, idx_atk = get_split(args,data,device)

from torch_geometric.utils import to_undirected
from utils import subgraph
data.edge_index = to_undirected(data.edge_index)
train_edge_index,_, edge_mask = subgraph(torch.bitwise_not(data.test_mask),data.edge_index,relabel_nodes=False)
mask_edge_index = data.edge_index[:,torch.bitwise_not(edge_mask)]
# filter out the unlabeled nodes except from training nodes and testing nodes, nonzero() is to get index, flatten is to get 1-d tensor
unlabeled_idx = (torch.bitwise_not(data.test_mask)&torch.bitwise_not(data.train_mask)).nonzero().flatten()

Namespace(add_edge_rate_1=0, add_edge_rate_2=0, cl_activation='relu', cl_base_model='GCNConv', cl_lr=0.0002, cl_num_epochs=0, cl_num_layers=2, cl_num_proj_hidden=32, cl_weight_decay=1e-05, config='config.yaml', cont_batch_size=0, cont_weight=100, cuda=True, dataset='Pubmed', debug=True, defense_mode='none', device_id=3, dis_weight=1, drop_edge_rate_1=0, drop_edge_rate_2=0.5, drop_feat_rate_1=0, drop_feat_rate_2=0.5, dropout=0.5, epochs=200, evaluate_mode='1by1', gpu_id=0, hidden=32, homo_boost_thrd=0.5, homo_loss_weight=0, inner=1, lr=0.01, model='GCN', no_cuda=False, noisy_level=0.25, num_hidden=32, num_proj_hidden=32, prune_thr=0.2, seed=10, selection_method='cluster_degree', target_class=0, target_loss_weight=1, tau=0.4, temperature=0.5, test_model='GCN', thrd=0.5, train_lr=0.01, trigger_size=3, trojan_epochs=400, use_vs_number=True, vs_number=160, vs_ratio=0, weight_decay=0.0005)


In [8]:
from torch_geometric.utils import to_dense_adj, dense_to_sparse
from torch_geometric.data import Data, InMemoryDataset

import os

import numpy as np
from scipy.linalg import expm

import torch
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.datasets import Planetoid, Amazon, Coauthor

# from seeds import development_seed
DATA_PATH = './data/'

def get_dataset(name: str, use_lcc: bool = True) -> InMemoryDataset:
    path = os.path.join(DATA_PATH, name)
    if name in ['Cora', 'Citeseer', 'Pubmed']:
        dataset = Planetoid(path, name)
    elif name in ['Computers', 'Photo']:
        dataset = Amazon(path, name)
    elif name == 'CoauthorCS':
        dataset = Coauthor(path, 'CS')
    else:
        raise Exception('Unknown dataset.')

    if use_lcc:
        lcc = get_largest_connected_component(dataset)

        x_new = dataset.data.x[lcc]
        y_new = dataset.data.y[lcc]

        row, col = dataset.data.edge_index.numpy()
        edges = [[i, j] for i, j in zip(row, col) if i in lcc and j in lcc]
        edges = remap_edges(edges, get_node_mapper(lcc))
        
        data = Data(
            x=x_new,
            edge_index=torch.LongTensor(edges),
            y=y_new,
            train_mask=torch.zeros(y_new.size()[0], dtype=torch.bool),
            test_mask=torch.zeros(y_new.size()[0], dtype=torch.bool),
            val_mask=torch.zeros(y_new.size()[0], dtype=torch.bool)
        )
        dataset.data = data

    return dataset


def get_component(dataset: InMemoryDataset, start: int = 0) -> set:
    visited_nodes = set()
    queued_nodes = set([start])
    row, col = dataset.data.edge_index.numpy()
    while queued_nodes:
        current_node = queued_nodes.pop()
        visited_nodes.update([current_node])
        neighbors = col[np.where(row == current_node)[0]]
        neighbors = [n for n in neighbors if n not in visited_nodes and n not in queued_nodes]
        queued_nodes.update(neighbors)
    return visited_nodes


def get_largest_connected_component(dataset: InMemoryDataset) -> np.ndarray:
    remaining_nodes = set(range(dataset.data.x.shape[0]))
    comps = []
    while remaining_nodes:
        start = min(remaining_nodes)
        comp = get_component(dataset, start)
        comps.append(comp)
        remaining_nodes = remaining_nodes.difference(comp)
    return np.array(list(comps[np.argmax(list(map(len, comps)))]))


def get_node_mapper(lcc: np.ndarray) -> dict:
    mapper = {}
    counter = 0
    for node in lcc:
        mapper[node] = counter
        counter += 1
    return mapper


def remap_edges(edges: list, mapper: dict) -> list:
    row = [e[0] for e in edges]
    col = [e[1] for e in edges]
    row = list(map(lambda x: mapper[x], row))
    col = list(map(lambda x: mapper[x], col))
    return [row, col]


def get_adj_matrix(data) -> np.ndarray:
    num_nodes = data.x.shape[0]
    adj_matrix = np.zeros(shape=(num_nodes, num_nodes))
    for i, j in zip(data.edge_index[0], data.edge_index[1]):
        adj_matrix[i, j] = 1.
    return adj_matrix


def get_ppr_matrix(
        adj_matrix: np.ndarray,
        alpha: float = 0.1) -> np.ndarray:
    num_nodes = adj_matrix.shape[0]
    A_tilde = adj_matrix + np.eye(num_nodes)
    D_tilde = np.diag(1/np.sqrt(A_tilde.sum(axis=1)))
    H = D_tilde @ A_tilde @ D_tilde
    return alpha * np.linalg.inv(np.eye(num_nodes) - (1 - alpha) * H)


def get_heat_matrix(
        adj_matrix: np.ndarray,
        t: float = 5.0) -> np.ndarray:
    num_nodes = adj_matrix.shape[0]
    A_tilde = adj_matrix + np.eye(num_nodes)
    D_tilde = np.diag(1/np.sqrt(A_tilde.sum(axis=1)))
    H = D_tilde @ A_tilde @ D_tilde
    return expm(-t * (np.eye(num_nodes) - H))


def get_top_k_matrix(A: np.ndarray, k: int = 128) -> np.ndarray:
    num_nodes = A.shape[0]
    row_idx = np.arange(num_nodes)
    A[A.argsort(axis=0)[:num_nodes - k], row_idx] = 0.
    norm = A.sum(axis=0)
    norm[norm <= 0] = 1 # avoid dividing by zero
    return A/norm


def get_clipped_matrix(A: np.ndarray, eps: float = 0.01) -> np.ndarray:
    num_nodes = A.shape[0]
    A[A < eps] = 0.
    norm = A.sum(axis=0)
    norm[norm <= 0] = 1 # avoid dividing by zero
    return A/norm


def set_train_val_test_split(
        seed: int,
        data: Data,
        num_development: int = 1500,
        num_per_class: int = 20) -> Data:
    rnd_state = np.random.RandomState(development_seed)
    num_nodes = data.y.shape[0]
    development_idx = rnd_state.choice(num_nodes, num_development, replace=False)
    test_idx = [i for i in np.arange(num_nodes) if i not in development_idx]

    train_idx = []
    rnd_state = np.random.RandomState(seed)
    for c in range(data.y.max() + 1):
        class_idx = development_idx[np.where(data.y[development_idx].cpu() == c)[0]]
        train_idx.extend(rnd_state.choice(class_idx, num_per_class, replace=False))

    val_idx = [i for i in development_idx if i not in train_idx]

    def get_mask(idx):
        mask = torch.zeros(num_nodes, dtype=torch.bool)
        mask[idx] = 1
        return mask

    data.train_mask = get_mask(train_idx)
    data.val_mask = get_mask(val_idx)
    data.test_mask = get_mask(test_idx)

    return data

class PPRDataset(InMemoryDataset):
    """
    Dataset preprocessed with GDC using PPR diffusion.
    Note that this implementations is not scalable
    since we directly invert the adjacency matrix.
    """
    def __init__(self,noisy_data,
                 name: str = 'Cora',
                 use_lcc: bool = True,
                 alpha: float = 0.1,
                 k: int = 16,
                 eps: float = None):
        self.name = name
        self.use_lcc = use_lcc
        self.alpha = alpha
        self.k = k
        self.eps = eps
        self.noisy_data = noisy_data

        super(PPRDataset, self).__init__(DATA_PATH)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self) -> list:
        return []

    @property
    def processed_file_names(self) -> list:
        return [str(self) + '.pt']

    def download(self):
        pass

    def process(self):
        # base = get_dataset(name=self.name, use_lcc=self.use_lcc)
        # generate adjacency matrix from sparse representation
        adj_matrix = get_adj_matrix(self.noisy_data)
        # obtain exact PPR matrix
        ppr_matrix = get_ppr_matrix(adj_matrix,
                                        alpha=self.alpha)

        if self.k:
            print(f'Selecting top {self.k} edges per node.')
            ppr_matrix = get_top_k_matrix(ppr_matrix, k=self.k)
        elif self.eps:
            print(f'Selecting edges with weight greater than {self.eps}.')
            ppr_matrix = get_clipped_matrix(ppr_matrix, eps=self.eps)
        else:
            raise ValueError

        # create PyG Data object
        edges_i = []
        edges_j = []
        edge_attr = []
        for i, row in enumerate(ppr_matrix):
            for j in np.where(row > 0)[0]:
                edges_i.append(i)
                edges_j.append(j)
                edge_attr.append(ppr_matrix[i, j])
        edge_index = [edges_i, edges_j]

        data = Data(
            x=self.noisy_data.x,
            edge_index=torch.LongTensor(edge_index),
            edge_attr=torch.FloatTensor(edge_attr),
            y=self.noisy_data.y,
            train_mask=torch.zeros(self.noisy_data.train_mask.size()[0], dtype=torch.bool),
            test_mask=torch.zeros(self.noisy_data.test_mask.size()[0], dtype=torch.bool),
            val_mask=torch.zeros(self.noisy_data.val_mask.size()[0], dtype=torch.bool)
        )

        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

    def __str__(self) -> str:
        return f'{self.name}_ppr_alpha={self.alpha}_k={self.k}_eps={self.eps}_lcc={self.use_lcc}'


# Unify Contrastive GNN

## Structure noise

### Noisy

In [9]:
args.num_hidden

32

In [10]:
import copy 
from model import UnifyModel, Encoder
from models.construct import model_construct
from construct_graph import *

import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

data = data.to(device)
num_class = int(data.y.max()+1)
# args.cl_activation = ({'relu': F.relu, 'prelu': nn.PReLU()})[config['activation']]
# args.cl_base_model = ({'GCNConv': GCNConv})[config['base_model']]

noisy_data = construct_noisy_graph(data,perturb_ratio=args.noisy_level,mode='random_noise')
noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



final_cl_acc_noisy = []
final_gnn_acc_noisy = []
print("=== Noisy graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=3)
for seed in seeds:
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
                            base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
    model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
    # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
    model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
    # model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
    # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
    acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
    print("Accuracy of GNN+CL: {}".format(acc_cl))
    final_cl_acc_noisy.append(acc_cl)
    gnn_model = model_construct(args,'GCN',noisy_data,device)
    gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
    print(clean_acc)
    final_gnn_acc_noisy.append(clean_acc)


print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
            .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

raw graph: torch.Size([2, 88648])
add edge: torch.Size([2, 110790])
remove edge: torch.Size([2, 66616])
updated graph: torch.Size([2, 88758])
=== Noisy graph ===
=== training gcn model ===
Epoch 0, training loss: 1059.32080078125 = 1.0883028507232666 + 100 * 10.58232593536377
Epoch 0, val loss: 1.0863984823226929


/home/project-robust-contrastive/Robust-Contrastive-Learning/model.py:226: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)


Epoch 10, training loss: 1059.3094482421875 = 1.0856796503067017 + 100 * 10.582237243652344
Epoch 10, val loss: 1.083805799484253
Epoch 20, training loss: 1059.28271484375 = 1.0830211639404297 + 100 * 10.58199691772461
Epoch 20, val loss: 1.0812100172042847
Epoch 30, training loss: 1059.2127685546875 = 1.0803284645080566 + 100 * 10.581324577331543
Epoch 30, val loss: 1.0786203145980835
Epoch 40, training loss: 1059.0184326171875 = 1.0774977207183838 + 100 * 10.5794095993042
Epoch 40, val loss: 1.0759011507034302
Epoch 50, training loss: 1058.4825439453125 = 1.0742725133895874 + 100 * 10.57408332824707
Epoch 50, val loss: 1.072792410850525
Epoch 60, training loss: 1057.058837890625 = 1.0705451965332031 + 100 * 10.559883117675781
Epoch 60, val loss: 1.0692049264907837
Epoch 70, training loss: 1053.510498046875 = 1.0663251876831055 + 100 * 10.524442672729492
Epoch 70, val loss: 1.0651582479476929
Epoch 80, training loss: 1045.495849609375 = 1.06161367893219 + 100 * 10.444342613220215
Epoc

### Raw

In [10]:
import copy 
from model import UnifyModel
from models.construct import model_construct


import os.path as osp
import random
from time import perf_counter as t
import yaml
from yaml import SafeLoader

import torch
import torch_geometric.transforms as T
from model import Encoder, Model, drop_feature

data = data.to(device)
config = yaml.load(open(args.config), Loader=SafeLoader)[args.dataset]
# num_epochs = config['num_epochs']
# args.cl_lr = config['args.cl_lr']
# weight_decay = config['weight_decay']
# args.seed = config['seed']
# args.cont_batch_size = config['cont_batch_size']
# args.cont_weight = config['cont_weight']
# args.add_edge_rate_1 = config['add_edge_rate_1']
# args.add_edge_rate_2 = config['add_edge_rate_2']
# args.drop_edge_rate_1 = config['drop_edge_rate_1']
# args.drop_edge_rate_2 = config['drop_edge_rate_2']
# args.drop_feat_rate_1 = config['drop_feature_rate_1']
# args.drop_feat_rate_2 = config['drop_feature_rate_2']
# args.add_edge_rate_1 = 0
# args.add_edge_rate_2 = 0
# args.drop_edge_rate_1 = 0.3
# args.drop_edge_rate_2 = 0.5
# args.drop_feat_rate_1 = 0.4
# args.drop_feat_rate_2 = 0.4
num_class = int(data.y.max()+1)

noisy_data = construct_noisy_graph(data,perturb_ratio=0.25,mode='random_noise')
noisy_data = noisy_data.to(device)
seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()


final_cl_acc = []
final_gnn_acc = []
print("=== Raw graph ===")
rs = np.random.RandomState(args.seed)
seeds = rs.randint(1000,size=3)
for seed in seeds:
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    '''Transductive'''
    encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
                            base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
    model = UnifyModel(args, encoder, args.num_hidden, args.num_proj_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=args.cl_weight_decay, device=device).to(device)
    model.fit(args, data.x, data.edge_index,data.edge_weight,data.y,idx_train,idx_val=idx_val,train_iters=args.cl_num_epochs,cont_iters=args.cl_num_epochs,seen_node_idx=None)
    # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
    acc_cl = model.test(data.x, data.edge_index,data.edge_weight,data.y,idx_clean_test)
    print("Accuracy of GNN+CL: {}".format(acc_cl))
    final_cl_acc.append(acc_cl)
    gnn_model = model_construct(args,'GCN',data,device)
    gnn_model.fit(data.x, data.edge_index, None, data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
    clean_acc = gnn_model.test(data.x,data.edge_index,data.edge_weight,data.y,idx_overall_test)
    print(clean_acc)
    final_gnn_acc.append(clean_acc)


print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
            .format(np.average(final_cl_acc),np.std(final_cl_acc),np.average(final_gnn_acc),np.std(final_gnn_acc)))

raw graph: torch.Size([2, 10556])
add edge: torch.Size([2, 13092])
remove edge: torch.Size([2, 7878])
updated graph: torch.Size([2, 10414])
=== Raw graph ===
=== training gcn model ===
Epoch 0, training loss: 861.644287109375 = 1.962094783782959 + 100 * 8.596821784973145
Epoch 0, val loss: 1.9662855863571167


/home/mfl5681/project-contrastive/RobustCL/model.py:224: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)


Epoch 10, training loss: 861.5540161132812 = 1.9520117044448853 + 100 * 8.596019744873047
Epoch 10, val loss: 1.9556394815444946
Epoch 20, training loss: 861.0609741210938 = 1.939118504524231 + 100 * 8.591217994689941
Epoch 20, val loss: 1.9420093297958374
Epoch 30, training loss: 858.0274658203125 = 1.9224451780319214 + 100 * 8.561050415039062
Epoch 30, val loss: 1.9245028495788574
Epoch 40, training loss: 839.6718139648438 = 1.9022588729858398 + 100 * 8.377695083618164
Epoch 40, val loss: 1.904028058052063
Epoch 50, training loss: 761.4630737304688 = 1.8792352676391602 + 100 * 7.59583854675293
Epoch 50, val loss: 1.8807604312896729
Epoch 60, training loss: 730.5070190429688 = 1.8613781929016113 + 100 * 7.28645658493042
Epoch 60, val loss: 1.8645652532577515
Epoch 70, training loss: 710.8011474609375 = 1.848585605621338 + 100 * 7.0895256996154785
Epoch 70, val loss: 1.8523708581924438
Epoch 80, training loss: 697.0316162109375 = 1.8360241651535034 + 100 * 6.951956272125244
Epoch 80, v

KeyboardInterrupt: 

### Noisy+DIffuse

In [ ]:
# import copy 
# from model import UnifyModel
# from models.construct import model_construct


# import os.path as osp
# import random
# from time import perf_counter as t
# import yaml
# from yaml import SafeLoader

# import torch
# import torch_geometric.transforms as T
# from model import Encoder, Model, drop_feature

# data = data.to(device)
# config = yaml.load(open(args.config), Loader=SafeLoader)[args.dataset]
# # num_epochs = config['num_epochs']
# # args.cl_lr = config['args.cl_lr']
# # weight_decay = config['weight_decay']
# args.seed = config['seed']
# # args.cont_batch_size = config['cont_batch_size']
# # args.cont_weight = config['cont_weight']
# # args.add_edge_rate_1 = config['add_edge_rate_1']
# # args.add_edge_rate_2 = config['add_edge_rate_2']
# # args.drop_edge_rate_1 = config['drop_edge_rate_1']
# # args.drop_edge_rate_2 = config['drop_edge_rate_2']
# # args.drop_feat_rate_1 = config['drop_feature_rate_1']
# # args.drop_feat_rate_2 = config['drop_feature_rate_2']
# # args.add_edge_rate_1 = 0
# # args.add_edge_rate_2 = 0
# # args.drop_edge_rate_1 = 0.3
# # args.drop_edge_rate_2 = 0.5
# # args.drop_feat_rate_1 = 0.4
# # args.drop_feat_rate_2 = 0.4
# num_class = int(data.y.max()+1)

# noisy_data = construct_noisy_graph(data,perturb_ratio=0.25,mode='random_noise')
# noisy_data = noisy_data.to(device)

# diff_dataset = PPRDataset(noisy_data,args.dataset)
# diff_noisy_data = diff_dataset.data.to(device)

# seen_node_idx = (torch.bitwise_not(data.test_mask)).nonzero().flatten()
# idx_overall_test = (torch.bitwise_not(data.train_mask)&torch.bitwise_not(data.val_mask)).nonzero().flatten()



# final_cl_acc_noisy = []
# final_gnn_acc_noisy = []
# print("=== Noisy graph ===")
# rs = np.random.RandomState(args.seed)
# seeds = rs.randint(1000,size=3)
# for seed in seeds:
#     np.random.seed(seed)
#     # torch.manual_seed(seed)
#     # torch.cuda.manual_seed(seed)
#     '''Transductive'''
#     encoder = Encoder(dataset.num_features, args.num_hidden, args.cl_activation,
#                             base_model=args.cl_base_model, k=args.cl_num_layers).to(device)
#     # model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device).to(device)
#     model = UnifyModel(args, encoder, args.cl_num_hidden, args.cl_num_proj_hidden, num_class, args.tau, lr=args.cl_lr, weight_decay=weight_decay, device=device,data1=noisy_data,data2=diff_noisy_data).to(device)
#     # model.fit(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
#     model.fit_1(args, noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_train,idx_val=idx_val,train_iters=num_epochs,cont_iters=num_epochs,seen_node_idx=None)
#     # x, edge_index,edge_weight,labels,idx_train,idx_val=None,cont_iters=None,train_iters=200,seen_node_idx = None
#     acc_cl = model.test(noisy_data.x, noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_clean_test)
#     print("Accuracy of GNN+CL: {}".format(acc_cl))
#     final_cl_acc_noisy.append(acc_cl)
#     gnn_model = model_construct(args,'GCN',noisy_data,device)
#     gnn_model.fit(noisy_data.x, noisy_data.edge_index, None, noisy_data.y, idx_train, idx_val,train_iters=args.epochs,verbose=False)
#     clean_acc = gnn_model.test(noisy_data.x,noisy_data.edge_index,noisy_data.edge_weight,noisy_data.y,idx_overall_test)
#     print(clean_acc)
#     final_gnn_acc_noisy.append(clean_acc)


# print('The final CL Acc:{:.5f}, {:.5f}, The final GNN Acc:{:.5f}, {:.5f}'\
#             .format(np.average(final_cl_acc_noisy),np.std(final_cl_acc_noisy),np.average(final_gnn_acc_noisy),np.std(final_gnn_acc_noisy)))

raw graph: torch.Size([2, 10556])
add edge: torch.Size([2, 13190])
remove edge: torch.Size([2, 7998])
updated graph: torch.Size([2, 10632])
=== Noisy graph ===
=== training gcn model ===
Epoch 0, training loss: 861.6275024414062 = 1.9418861865997314 + 100 * 8.596856117248535
Epoch 0, val loss: 1.9364506006240845
Epoch 10, training loss: 848.9478759765625 = 1.897546648979187 + 100 * 8.470503807067871
Epoch 10, val loss: 1.8890513181686401
Epoch 20, training loss: 760.0466918945312 = 1.8456153869628906 + 100 * 7.582010746002197
Epoch 20, val loss: 1.8392390012741089
Epoch 30, training loss: 731.3434448242188 = 1.8136775493621826 + 100 * 7.295298099517822
Epoch 30, val loss: 1.808591365814209
Epoch 40, training loss: 714.7428588867188 = 1.788662314414978 + 100 * 7.129542350769043
Epoch 40, val loss: 1.7784889936447144
Epoch 50, training loss: 705.1812744140625 = 1.7627829313278198 + 100 * 7.034185409545898
Epoch 50, val loss: 1.748166561126709
Epoch 60, training loss: 695.6590576171875 = 

KeyboardInterrupt: 